In [1]:
import yt
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import pickle as pickle
from unyt import g,cm
from numpy.linalg import eig


In [105]:
box_A_root = '/data/gnedin/REI/D/Cai.B40.N256L2.sf=1_uv=0.15_bw=10_res=100.WC1/C/'
hc = pd.read_csv(box_A_root+'a=0.1453/hpropsRS.res', names=['Mvir','Mstar','Z','SFR','luminosity','px','py','pz','Rvir'],delim_whitespace=True,index_col=False)
ds = yt.load('/data/gnedin/REI/D/Cai.B40.N256L2.sf=1_uv=0.15_bw=10_res=100.WC1/C/rei40c1_a0.1453/rei40c1_a0.1453.art')
p = ds.artio_parameters
abox = p["abox"][0]
auni = p["auni"][0]
scale = auni/abox
##difference is llyman - the galaxies distances
data = []

yt : [WARNING  ] 2023-03-07 12:53:54,300 DeltaDC != 0, which implies auni != abox. Be sure you understand which expansion parameter is appropriate for your use! (Gnedin, Kravtsov, & Rudd 2011)
yt : [INFO     ] 2023-03-07 12:53:54,401 Parameters: current_time              = 958007400.6400309 yr
yt : [INFO     ] 2023-03-07 12:53:54,402 Parameters: domain_dimensions         = [256 256 256]
yt : [INFO     ] 2023-03-07 12:53:54,403 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-03-07 12:53:54,404 Parameters: domain_right_edge         = [256. 256. 256.]
yt : [INFO     ] 2023-03-07 12:53:54,405 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2023-03-07 12:53:54,406 Parameters: current_redshift          = 5.881049592936443
yt : [INFO     ] 2023-03-07 12:53:54,407 Parameters: omega_lambda              = 0.6964000165462494
yt : [INFO     ] 2023-03-07 12:53:54,407 Parameters: omega_matter              = 0.3035999834537506
yt : [INFO     ] 2023-03-07 12:53:5

In [2]:
with open('vector.pkl', 'rb') as f:
    vector = pickle.load(f)
with open('peak_locations_box_c.pkl', 'rb') as f:
    peaks = pickle.load(f)
with open('lyman_cul.pkl', 'rb') as f:
    integral = pickle.load(f)
with open('angle_3.pkl', 'rb') as f:
    angle = pickle.load(f)

In [4]:
peaks[1885]

[33.828195108952556, 0.019444643898385294, 7.9983927030848605]

In [107]:
clean_vector_data =[]
for i in vector:
    clean_vector_data.append([i[0][0],i[0][1],i[0][2]])

In [108]:
len(clean_vector_data)

2895

In [118]:
starting_locations =[]

In [119]:
for i,z in zip(peaks,clean_vector_data):
    starting_locations.append(np.subtract(i,np.multiply(z,2)))


In [110]:
def convertCartToSphere(x,y,z):
    r = np.sqrt(x**2+y**2+z**2)
    theta = np.arccos(z/r)
    phi = np.arctan2(y,x)
    return [theta,phi]

In [111]:
sphereical = []
for i in clean_vector_data:
    sphereical.append(convertCartToSphere(i[0],i[1],i[2]))

In [121]:
data = {
    "direction": clean_vector_data,
    "direction_spherical": sphereical,
    "peaks": peaks,
    "column_density" : integral,
    "angle_respect_to_los" : angle,
}

In [122]:
df_1 = pd.DataFrame(data)

In [128]:
new_df  = df_1.sort_values(by = ['angle_respect_to_los'])

In [129]:
new_df.to_csv('check_out_new.csv')

index: 110,189  angles: 0.985, 0.032  column density: 1.337209152877845e+19 6.235100603471154e+20

**peaks_location: 
1: [16.89958306571362, 20.797551467597238, 10.458671121625699]
2: [5.0499879620436285, 8.774408530526296, 28.93790342729936]

**least eigen value eigen vector:
1: [ 0.457119  , -0.23058705 , 0.85899466]
2: [-0.72362639, -0.68210974, -0.10531454]

**starting points 1 chimp away:
1: [16.054518385559316, 20.550711340107522, 9.359587912378187](scaling the center)
    [16.44246406851914, 21.02813852139452, 9.599676457483366] (without scaling from yt)
2: [5.657687169640689, 9.255093542238283, 28.378921423321636] (scaling the center)
    [5.773614350876813, 9.456518269505649, 29.043217972229133] (without scaling from yt)

**directions:
theta1 = 0.5371935 phi1 = -0.467189
theta2 = 1.676307 phi2 = -2.3857197

In [ ]:
angle[189]

0.032262369393398216

In [ ]:
def convertSphereToCart(theta, phi):
    "converts a unit vector in spherical to cartesian, needed for getGalaxies"
    return np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])

In [ ]:
convertSphereToCart(0.6217380158991004, 0.561899686523328)

array([0.4928941 , 0.31032568, 0.81286738])

In [ ]:
convertSphereToCart(0.5371935,-0.467189)

array([ 0.45688905, -0.23047065,  0.85914822])

In [ ]:
convertSphereToCart(1.676307,-2.3857197)

array([-0.72362639, -0.68210967, -0.10531502])

In [ ]:
sp = ds.sphere(ds.arr((peaks[110][0]/scale,peaks[110][1]/scale,peaks[110][2]/scale), 'Mpccm/h'), (3/scale, "Mpccm/h"))

In [ ]:
def unit_vector(vector):
    return vector/np.linalg.norm(vector)

In [ ]:
sample_vector = unit_vector(vector[189])

In [ ]:
new_vector = sample_vector*1

In [ ]:
peaks[189][0]/scale - new_vector[0][0], peaks[189][1]/scale - new_vector[0][1],peaks[189][2]/scale - new_vector[0][2], 

(5.657687169640689, 9.255093542238283, 28.378921423321636)

In [ ]:
x = x*6.5808248e+23

In [ ]:
x *cm

unyt_array([-7.78475939e+21, -4.90127737e+21,  1.28384107e+22], 'cm')

In [ ]:
sp

YTSphere (rei40c1_a0.1453.art): , center=[1.28943979e+24 7.82275392e+24 1.62002336e+25] cm, radius=3.2149771006435e+23 cm

In [ ]:
sp[('gas','x')].to('Mpccm/h')

unyt_array([2.15636679, 2.15636679, 2.15636679, ..., 1.81287474,
            1.77470895, 1.81287474], 'Mpccm/h')

In [ ]:
sp[('gas','y')].to('Mpccm/h')

unyt_array([12.23213374, 12.23213374, 12.27029952, ..., 12.30846531,
            12.23213374, 12.23213374], 'Mpccm/h')

In [ ]:
sp[('gas','z')].to('Mpccm/h')

unyt_array([24.36885302, 24.40701881, 24.40701881, ..., 24.78867664,
            24.903174  , 24.903174  ], 'Mpccm/h')

In [ ]:
distance = (np.sqrt((sp[('gas','x')].to('Mpccm/h') - sp.center.to('Mpccm/h')[0])**2 + (sp[('gas','y')].to('Mpccm/h') - sp.center.to('Mpccm/h')[1])**2 + (sp[('gas','z')].to('Mpccm/h') - sp.center.to('Mpccm/h')[2])**2))

In [ ]:
for i in distance:
    print(i)

0.46842786965848865 Mpccm/h
0.44941364992212696 Mpccm/h
0.4793610977711258 Mpccm/h
0.4857175193002228 Mpccm/h
0.4674071760453624 Mpccm/h
0.48772898398896875 Mpccm/h
0.47561927568185397 Mpccm/h
0.45048323700221193 Mpccm/h
0.4803640079564681 Mpccm/h
0.48068442703622855 Mpccm/h
0.45582778525996737 Mpccm/h
0.4853796484557399 Mpccm/h
0.4272819354497588 Mpccm/h
0.40634691093353953 Mpccm/h
0.4586766679063832 Mpccm/h
0.43924053244804334 Mpccm/h
0.43291301130447407 Mpccm/h
0.4122640355648863 Mpccm/h
0.4639268354780939 Mpccm/h
0.44472021355033603 Mpccm/h
0.4729269846256311 Mpccm/h
0.47802066459736625 Mpccm/h
0.4642585992007707 Mpccm/h
0.47561158330149605 Mpccm/h
0.44178134011472536 Mpccm/h
0.42156697230345347 Mpccm/h
0.4722130724509576 Mpccm/h
0.45335763526945416 Mpccm/h
0.45369712773198895 Mpccm/h
0.4340380667358507 Mpccm/h
0.48337926831036865 Mpccm/h
0.46497685608619865 Mpccm/h
0.4860666959368464 Mpccm/h
0.4857684063815381 Mpccm/h
0.46118585847607996 Mpccm/h
0.4787084197568689 Mpccm/h
0.453743

In [ ]:
sp[('gas','density')]

unyt_array([1.74957956e-28, 1.32053151e-28, 1.27733387e-28, ...,
            1.26035566e-28, 2.04770167e-28, 2.62905225e-28], 'g/cm**3')

In [ ]:
for i in sp[('gas','dx')]:
    print(i)

2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm
2.511700860152598e+22 cm


In [ ]:
sp.center.to('cm')

unyt_array([1.28943979e+24, 7.82275392e+24, 1.62002336e+25], 'cm')

In [ ]:
sp.center.to('Mpccm/h') 

unyt_array([ 1.95932888, 11.88682698, 24.61657055], 'Mpccm/h')

In [ ]:
x[0]

-7.784759391270598e+21

In [ ]:
start = [sp.center.to('cm')[0] - x[0]*cm,sp.center.to('cm')[1] - x[1]*cm,sp.center.to('cm')[2] - x[2]*cm]
end = [sp.center.to('cm')[0] + x[0]*cm,sp.center.to('cm')[1] + x[1]*cm,sp.center.to('cm')[2] + x[2]*cm]

In [ ]:
start

[unyt_quantity(1.29722455e+24, 'cm'),
 unyt_quantity(7.8276552e+24, 'cm'),
 unyt_quantity(1.61873952e+25, 'cm')]

In [ ]:
end

[unyt_quantity(1.28165503e+24, 'cm'),
 unyt_quantity(7.81785264e+24, 'cm'),
 unyt_quantity(1.6213072e+25, 'cm')]

In [ ]:
dx = end[0] - start[0]
dy = end[1] - start[1]
dz = end[2] - start[2]

In [ ]:
dx = np.linspace(end[0],start[0], 1000)
dy = np.linspace(end[1],start[1], 1000)
dz = np.linspace(end[2],start[2], 1000)

In [ ]:
def finding_density(start, end,):
    sp = ds.sphere(ds.arr((peaks[0][0]/scale,peaks[0][1]/scale,peaks[0][2]/scale), 'Mpccm/h'), (0.5/scale, "Mpccm/h"))
    

In [ ]:
location = []
for i,j,k in zip(dx,dy,dz):
    location.append([i,j,k])

In [ ]:
location

[[unyt_quantity(1.28165503e+24, 'cm'),
  unyt_quantity(7.81785264e+24, 'cm'),
  unyt_quantity(1.6213072e+25, 'cm')],
 [unyt_quantity(1.28167062e+24, 'cm'),
  unyt_quantity(7.81786245e+24, 'cm'),
  unyt_quantity(1.62130463e+25, 'cm')],
 [unyt_quantity(1.2816862e+24, 'cm'),
  unyt_quantity(7.81787227e+24, 'cm'),
  unyt_quantity(1.62130206e+25, 'cm')],
 [unyt_quantity(1.28170179e+24, 'cm'),
  unyt_quantity(7.81788208e+24, 'cm'),
  unyt_quantity(1.62129949e+25, 'cm')],
 [unyt_quantity(1.28171737e+24, 'cm'),
  unyt_quantity(7.81789189e+24, 'cm'),
  unyt_quantity(1.62129692e+25, 'cm')],
 [unyt_quantity(1.28173296e+24, 'cm'),
  unyt_quantity(7.8179017e+24, 'cm'),
  unyt_quantity(1.62129435e+25, 'cm')],
 [unyt_quantity(1.28174854e+24, 'cm'),
  unyt_quantity(7.81791151e+24, 'cm'),
  unyt_quantity(1.62129178e+25, 'cm')],
 [unyt_quantity(1.28176413e+24, 'cm'),
  unyt_quantity(7.81792133e+24, 'cm'),
  unyt_quantity(1.62128921e+25, 'cm')],
 [unyt_quantity(1.28177971e+24, 'cm'),
  unyt_quantity(7.81

In [ ]:
location[0]

[unyt_quantity(1.28165503e+24, 'cm'),
 unyt_quantity(7.81785264e+24, 'cm'),
 unyt_quantity(1.6213072e+25, 'cm')]

In [ ]:
for i,j,k,p,q in zip(sp['gas','x'],sp['gas','y'],sp['gas','z'],sp['gas','dx'],sp['gas','density']):
    if(np.sqrt(np.square(i-location[0][0])+np.square(j-location[0][1])+np.square(k-location[0][2]))<p):
        print(q)

8.22931823242855e-27 g/cm**3
9.506147863907385e-27 g/cm**3
1.0035320823575686e-26 g/cm**3
8.570154905058161e-27 g/cm**3
1.2108978178781408e-26 g/cm**3
1.7314663506519475e-26 g/cm**3
1.2767496276325931e-26 g/cm**3
1.081834942745004e-26 g/cm**3


In [ ]:
sp[('gas','density')]

yt : [INFO     ] 2023-03-02 16:05:26,451 Created 48 chunks for ARTIO


unyt_array([1.74957956e-28, 1.32053151e-28, 1.27733387e-28, ...,
            1.26035566e-28, 2.04770167e-28, 2.62905225e-28], 'g/cm**3')